<a href="https://colab.research.google.com/github/ohmono/clustering-spark/blob/main/CreacionTablasHive_en_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<p><img alt="udeA logo" height="120px" src="https://github.com/freddyduitama/images/blob/master/logo.png?raw=true" align="left" hspace="10px" vspace="0px" style="width:107px;height:152px;"></p>

# <center> <font color='0B5345'>Creación de tablas y particiones con HIVE.</font> </center>
##### <center><font face="Verdana"><a  href="https://colab.research.google.com/drive/13L6BpqgXtFdd9B-bT3Sr9ZksQ5U1dRlO#scrollTo=QtHm4ClUKJTI">El diseño físico y el Big Data.</a><a>&nbsp;&nbsp; | </a><a  href="https://colab.research.google.com/drive/1Ddb-oi4v_FaJPOi2LrdAENitLC8GWpu5#scrollTo=-O6sHpqcFnVCb" >TOC</a><a>&nbsp;  |</a> <a>&nbsp;&nbsp;</a> <a   href="https://colab.research.google.com/drive/1ICKacknjauqwsLGirbURae5IUdHX759d">Map Reduce.</a></font><center><br>

## 1. Introducción

<font face="Verdana" size="2">
<p align="justify">
<em>Apache Hive™</em> es una bodega de datos construida sobre <em>Apache Hadoop</em> para proporcionar consultas y análisis de datos[3].
El software de almacén de datos <em>Apache Hive™</em> facilita la lectura, escritura y gestión de grandes conjuntos de datos que residen en el almacenamiento distribuido utilizando SQL. Hive ofrece una interfaz similar a SQL para consultar datos almacenados en varias bases de datos y sistemas de archivos que se integran con Hadoop.  Aunque inicialmente fue desarrollado por <em>Facebook™</em>, Apache Hive es utilizado y desarrollado por otras empresas como Netflix y la Autoridad Reguladora de la Industria Financiera (FINRA). <em>Amazon™</em> mantiene un fork de software de Apache Hive incluido en <em>Amazon Elastic MapReduce</em> en Amazon Web Services.
</p>

## 2. Ejemplos de SQL con HIVE

### 2.1 Configurando el ambiente de trabajo.

<font face="Verdana" size="2">
<p align="justify">
<b>Paso 1:</b> Verifique la librería de SPARK más reciente <a href="http://apache.osuosl.org/spark/">Acá</a>. Si esta es diferente a la especificada abajo, debe cambiar la versión de SPARK y de HADOOP en las dos siguientes celdas

- Las instrucciones:  !wget -q......tar...
- También debe actualizar las variables de ambiente de JAVA.
</p>

In [ ]:
# Estas intrucciones instalan el ambiente de spark y de hadoop (el DFS que vamos a usar)..solo se corre una vez
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
#En esta celda se configuran las variables de ambiente necesarias para correr los programas
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

<font face="Verdana" size="2">
<p align="justify">
<b>Paso 2:</b>  
A continuación se importan las librerías de SPARK y de HIVE.

- SPARK es el framework que implementa una versión extendida de Map-Reduce, el paradigma de programación paralela y distribuida que usaremos.
- HIVE  es el la bodega de datos basada en SQL de almacenamiento distribuido de la información que corre sobre HADOOP.</p>

In [ ]:
#importa pyspark package
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext

<font face="Verdana" size="2">
<p align="justify">
<b>Paso 3:</b>
Acá se configura la sesión de trabajo.

In [ ]:
# define  la sesion SPARK.
conf = SparkConf().setAppName("ejemplo").setMaster("local[*]")
sc = SparkContext(conf=conf)
sqlContextHive = HiveContext(sc)          # deprecated


/content/spark-3.2.2-bin-hadoop3.2/python/pyspark/sql/context.py:604: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  FutureWarning


In [ ]:
# crea la sesión SPARK.
spark = SparkSession.builder
spark = spark.enableHiveSupport()
spark = spark.config(conf)

## 2.2 Creando la Base de Datos en HIVE.
<font face="Verdana" size="2">
<p align="justify">
<b>Paso 4:</b>
Acá se crea la base de datos de trabajo llamada <em>test</em>. Por defecto HIVE tiene una base de datos llamada <em>default</em>.
</p>

In [ ]:
#Acá se crea la base de datos de prueba. Solo se crea una vez la base de datos.
sqlContextHive.sql('DROP DATABASE IF EXISTS test')
sqlContextHive.sql('CREATE DATABASE test')

DataFrame[]

In [ ]:
sqlContextHive.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
|     test|
+---------+



<font face="Verdana" size="2">
<p align="justify">
<b>Paso 5:</b>
Para poder usar google drive como su directorio de trabajo, ejecute la siguiente instrucción e ingrese el <em>código de autorización solicitado</em>.
</p>

In [ ]:
# mount your google driver
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


<font face="Verdana" size="2">
<p align="justify">
<b>Paso 6:</b>
Descargue los archivo con los los datos que vamos a usar desde <a href="https://drive.google.com/drive/u/0/folders/1D235z6ThQzo3JYE0bLIl1vXzxM9Yl_PP">acá</a>. Súbalo a su google drive.

In [ ]:
#Verifique el path de los archivos de datos a ser cargados posteriormente en la base de datos.
# Debe adecuar esta instrucción a la ubicación que le asigne en su drive.
!ls -l '/gdrive/My Drive/Colab Notebooks/BdeD/Data/'

total 15480
-rw------- 1 root root 10327345 Jul 13  2021  auth.log
-rw------- 1 root root     8910 Jul 13  2021  barrios.csv
-rw------- 1 root root     2505 Mar 31  2021 'base de datos (1).rar'
-rw------- 1 root root  3602146 Jul 13  2021  clientes.csv
-rw------- 1 root root  1899304 Jul 13  2021  dispositivos.csv
drwx------ 2 root root     4096 Aug 31 14:16  file01
drwx------ 2 root root     4096 Aug 30 03:29  file02
-rw------- 1 root root      528 Mar 18  2021  sem2020-2.csv
-rw------- 1 root root      416 Mar 18  2021  sem2021-1.csv
-rw------- 1 root root      416 Mar 18  2021  sem2021-2.csv


In [ ]:
# Por precaución borramos las tablas a ser creadas
sqlContextHive.sql("DROP TABLE IF EXISTS test.alumnos_grupo")
sqlContextHive.sql("DROP TABLE IF EXISTS test.alumnos_grupo_paticionado")

DataFrame[]

## 2.3 Creando una tabla sin especificar las particiones.
<font face="Verdana" size="2">
<p align="justify">
<b>Paso 7:</b>
Vamos a proceder a crear una tabla HIVE. No se especifica ningún tipo de partición. Este sentencia es equivalente a la partición <em>round-robin</em>, es decir no conocemos donde queda ubicado cada registro. Hadoop particiona el archivo de la BdeD, pero no hay control como se ubican los registros por partición. Note que puedo indicar los archivos en Hadoop que deseo usar con la tabla.<br>
</p>

In [ ]:
# Borra la tabla si existe y luego la crea de nuevo, indicando el archivos Hadoop DFS a usar.
sqlContextHive.sql("DROP TABLE IF EXISTS test.alumnos_grupo")
sqlContextHive.sql("CREATE TABLE IF NOT EXISTS test.alumnos_grupo( \
        materia             STRING COMMENT 'código de la materia',\
        grupo               INT,\
        alumno              INT        COMMENT 'Cédula del estudiante',\
        nota                DOUBLE,\
        semestre            STRING) \
        ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' \
        LOCATION '/gdrive/My Drive/Colab Notebooks/BdeD/Data/file01';")

DataFrame[]

In [ ]:
# muestra la descripción de la tabla.
sqlContextHive.sql("DESCRIBE test.alumnos_grupo" ).show()

+--------+---------+--------------------+
|col_name|data_type|             comment|
+--------+---------+--------------------+
| materia|   string|código de la materia|
|   grupo|      int|                null|
|  alumno|      int|Cédula del estudi...|
|    nota|   double|                null|
|semestre|   string|                null|
+--------+---------+--------------------+



<font face="Verdana" size="2">
<p align="justify">
<b>Paso 8:</b>
Vamos a proceder a cargar los datos desde un archivo <em>csv</em> ubicado en el path especificado. Note que hay archivos que no indican el semetre al que corresponde la información.<br>
</p>

In [ ]:
# cargamos los datos desde el archivo indicado
sqlContextHive.sql("LOAD DATA LOCAL INPATH '/gdrive/My Drive/Colab Notebooks/BdeD/Data/sem2020-2.csv' INTO TABLE test.alumnos_grupo" )
sqlContextHive.sql("LOAD DATA LOCAL INPATH '/gdrive/My Drive/Colab Notebooks/BdeD/Data/sem2021-1.csv' INTO TABLE test.alumnos_grupo")
sqlContextHive.sql("LOAD DATA LOCAL INPATH '/gdrive/My Drive/Colab Notebooks/BdeD/Data/sem2021-2.csv' INTO TABLE test.alumnos_grupo")

DataFrame[]

In [ ]:
# muestra los datos cargados en la tabla. Los atributos en  null no fueron especificados en los archivos de carga, es decir no tienen semestre.
sqlContextHive.sql("SELECT * FROM test.alumnos_grupo").show(50)

+-------+-----+----------+----+--------+
|materia|grupo|    alumno|nota|semestre|
+-------+-----+----------+----+--------+
| IIN101|    1|1018999677| 3.5|  2020-2|
| IIN101|    1|1019000786| 4.8|  2020-2|
| IIN101|    1|1078499677| 3.5|  2020-2|
| IIN101|    1|1019000786| 3.9|  2020-2|
| IEO121|    1|1018999677| 3.8|  2020-2|
| IEO121|    1|1089430786| 3.8|  2020-2|
| IEO121|    1|1018499677| 5.0|  2020-2|
| IEO121|    1|1039030786| 2.8|  2020-2|
| IEO101|    1|1068939677| 4.5|  2020-2|
| IEO101|    1|1029030786| 3.8|  2020-2|
| IEO101|    1|1978039677| 4.5|  2020-2|
| ISI101|    1|1039530786| 3.8|  2020-2|
| ISI101|    1|1028949677| 2.5|  2020-2|
| ISI101|    1|1059035786| 4.8|  2020-2|
| ISI101|    1|1038059677| 4.5|  2020-2|
| ISI101|    1|1079564786| 3.7|  2020-2|
| ISI101|    1|1018999677| 3.5|    null|
| ISI101|    1|1019000786| 3.8|    null|
| ISI101|    1|1078499677| 5.5|    null|
| ISI101|    1|1019000786| 2.9|    null|
| ISI101|    2|1018999677| 3.8|    null|
| ISI101|    2|1

In [ ]:
# muestra la ubicación en disco de los archivos donde se almacena la tabla HIVE.
# Note que los nombres de los archivos no están asociados a particiones lógicas, muchos registros no tienen semestre asociado.
!ls -l '/gdrive/My Drive/Colab Notebooks/BdeD/Data/file01'

total 8
-rw------- 1 root root 528 Aug 31 14:12 sem2020-2_copy_1.csv
-rwx------ 1 root root 528 Aug 31 14:16 sem2020-2_copy_2.csv
-rwx------ 1 root root 528 Aug 31 14:20 sem2020-2_copy_3.csv
-rw------- 1 root root 528 Aug 30 13:26 sem2020-2.csv
-rw------- 1 root root 416 Aug 31 14:12 sem2021-1_copy_1.csv
-rwx------ 1 root root 416 Aug 31 14:16 sem2021-1_copy_2.csv
-rwx------ 1 root root 416 Aug 31 14:20 sem2021-1_copy_3.csv
-rw------- 1 root root 416 Aug 30 13:26 sem2021-1.csv
-rw------- 1 root root 416 Aug 31 14:12 sem2021-2_copy_1.csv
-rwx------ 1 root root 416 Aug 31 14:16 sem2021-2_copy_2.csv
-rwx------ 1 root root 416 Aug 31 14:20 sem2021-2_copy_3.csv
-rw------- 1 root root 416 Aug 30 13:26 sem2021-2.csv


## 2.4. Creando particiones en HIVE
<font face="Verdana" size="2">
<p align="justify">
<b>Paso 9:</b> Vamos a proceder a crear una tabla HIVE especificando las particiones deseadas. Suponga que tiene una bodega de datos como la de la figura 1.

<center><img src="https://github.com/freddyduitama/figuras/blob/main/bodega.png?raw=true"  height="300" width="600"></center>
<caption><center><font color='0B5345'> <u> <b>Figura 1: Bodega de datos con la historia académica.</b><br> </u> .</font></center></caption>


<font face="Verdana" size="2">
<p align="justify">
Vamos a proceder a crear una tabla HIVE para almacenar los hechos de la bodega especificando las particiones deseadas. La operación que vamos a realizar en el ejemplo consta de dos pasos

- Nuestro propósito es crear una partición por cada semestre académico; es decir, en cada partición se almacenará la información de los alumnos matriculados en los grupos abiertos por cada semestre académico cursado. Se define una partición por lista de valores. Es decir, indicamos el valor o los valores asociados a una columna que serán almacenados en cada partición.
- Dentro de cada partición previa queremos almacenar los alumnos de un mismo grupo en una misma cubeta, usamos entonces una partición hash. Dentro de cada partición creada previamente, se crean cinco cubetas para almacenar allí los registros. HIVE  usa una function hash que los ubica según la materia y el grupo de cada estudiante matriculado.

<center><img src="https://github.com/freddyduitama/figuras/blob/main/particion-hive.png?raw=true"  height="150" width="600"></center>
<caption><center><font color='0B5345'> <u> <b>Figura 2: ¨Particiones requeridas sobre la tabla Alumnos_grupo.</b><br> </u> .</font></center></caption>

- La sintaxis  para de la creación de las particiones la encuentra en la siguiente celda. Para consultar todas las variantes que ofrece HIVE para crear particiones puede leer <a href="https://spark.apache.org/docs/latest/sql-ref-syntax-ddl-create-table-hiveformat.html">acá </a>

In [ ]:
sqlContextHive.sql("DROP TABLE IF EXISTS test.alumnos_grupo_particionado")
sqlContextHive.sql("CREATE TABLE IF NOT EXISTS test.alumnos_grupo_particionado( \
        materia             STRING COMMENT 'código de la materia',\
        grupo               INT,\
        alumno              INT        COMMENT 'Cédula del estudiante',\
        nota                DOUBLE)  \
        PARTITIONED BY (semestre  STRING) \
        CLUSTERED BY (materia,grupo) \
        INTO 5 BUCKETS \
        ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' \
        LOCATION '/gdrive/My Drive/Colab Notebooks/BdeD/Data/file02';")

DataFrame[]

<font face="Verdana" size="2">
<p align="justify">
<b>Paso 9:</b> Vamos a proceder a cargar los datos indicando la partición destino de acda archivo cargado.
</p>

In [ ]:
# cargamos los datos desde el archivo indicado
sqlContextHive.sql("LOAD DATA LOCAL INPATH '/gdrive/My Drive/Colab Notebooks/BdeD/Data/sem2020-2.csv' INTO TABLE test.alumnos_grupo_particionado PARTITION(semestre='2020-2')")
sqlContextHive.sql("LOAD DATA LOCAL INPATH '/gdrive/My Drive/Colab Notebooks/BdeD/Data/sem2021-1.csv' INTO TABLE test.alumnos_grupo_particionado PARTITION(semestre='2021-1')")
sqlContextHive.sql("LOAD DATA LOCAL INPATH '/gdrive/My Drive/Colab Notebooks/BdeD/Data/sem2021-2.csv' INTO TABLE test.alumnos_grupo_particionado PARTITION(semestre='2021-2')")

DataFrame[]

In [ ]:
sqlContextHive.sql("SELECT * FROM test.alumnos_grupo_particionado").show(50)

+-------+-----+----------+----+--------+
|materia|grupo|    alumno|nota|semestre|
+-------+-----+----------+----+--------+
| IIN101|    1|1018999677| 3.5|  2020-2|
| IIN101|    1|1019000786| 4.8|  2020-2|
| IIN101|    1|1078499677| 3.5|  2020-2|
| IIN101|    1|1019000786| 3.9|  2020-2|
| IEO121|    1|1018999677| 3.8|  2020-2|
| IEO121|    1|1089430786| 3.8|  2020-2|
| IEO121|    1|1018499677| 5.0|  2020-2|
| IEO121|    1|1039030786| 2.8|  2020-2|
| IEO101|    1|1068939677| 4.5|  2020-2|
| IEO101|    1|1029030786| 3.8|  2020-2|
| IEO101|    1|1978039677| 4.5|  2020-2|
| ISI101|    1|1039530786| 3.8|  2020-2|
| ISI101|    1|1028949677| 2.5|  2020-2|
| ISI101|    1|1059035786| 4.8|  2020-2|
| ISI101|    1|1038059677| 4.5|  2020-2|
| ISI101|    1|1079564786| 3.7|  2020-2|
| ISI101|    1|1018999677| 3.5|  2021-1|
| ISI101|    1|1019000786| 3.8|  2021-1|
| ISI101|    1|1078499677| 5.5|  2021-1|
| ISI101|    1|1019000786| 2.9|  2021-1|
| ISI101|    2|1018999677| 3.8|  2021-1|
| ISI101|    2|1

In [ ]:
sqlContextHive.sql("Show partitions test.alumnos_grupo_particionado").show(truncate=False)

+---------------+
|partition      |
+---------------+
|semestre=2020-2|
|semestre=2021-1|
|semestre=2021-2|
+---------------+



In [ ]:
# muestra la ubicación en disco de los archivos donde se almacena la tabla HIVE
!ls -l '/gdrive/My Drive/Colab Notebooks/BdeD/Data/file02'

total 12
drwx------ 2 root root 4096 Aug 31 14:20 'semestre=2020-2'
drwx------ 2 root root 4096 Aug 31 14:20 'semestre=2021-1'
drwx------ 2 root root 4096 Aug 31 14:20 'semestre=2021-2'


In [ ]:
# muestra la descripción de la tabla.
sqlContextHive.sql("DESCRIBE FORMATTED test.alumnos_grupo_particionado" ).show(30)

+--------------------+--------------------+--------------------+
|            col_name|           data_type|             comment|
+--------------------+--------------------+--------------------+
|             materia|              string|código de la materia|
|               grupo|                 int|                null|
|              alumno|                 int|Cédula del estudi...|
|                nota|              double|                null|
|            semestre|              string|                null|
|# Partition Infor...|                    |                    |
|          # col_name|           data_type|             comment|
|            semestre|              string|                null|
|                    |                    |                    |
|# Detailed Table ...|                    |                    |
|            Database|                test|                    |
|               Table|alumnos_grupo_par...|                    |
|               Owner|   

In [ ]:
# Necesario para borrar BdeD test
sqlContextHive.sql("DROP TABLE IF EXISTS test.alumnos_grupo")
sqlContextHive.sql("DROP TABLE IF EXISTS test.alumnos_grupo")
sqlContextHive.sql("DROP TABLE IF EXISTS test.alumnos_grupo_particionado")

DataFrame[]

# Bibliografía
<p align="justify">
[1]&nbsp;&nbsp;&nbsp;&nbsp;Apache Software Foundation. <cite>Apache Hive 2.0. </cite><a href="https://hive.apache.org/"> Click</a> </p>



<p align="left"><b><font face='Courier New' color="black" align="left" size=4>Copyright.</font></b>
<img alt="udeA logo" height="120px" src="https://github.com/freddyduitama/images/blob/master/in2lab.png?raw=true" align="right" hspace="10px" vspace="0px" height="120" width="350"">
                                                                                                                              
<font face='Verdana' size=2>
John Freddy Duitama M.<br>
Universidad de Antioquia.<br>
Apartado Aéreo 1226 | Dirección: calle 67 No. 53 - 108.<br>
Medellín, Colombia. South America.
    
</p>
</font>